### LAX Prior Elicitation 4 Block Tower Results Visualizer
Author : Catherine Wong, adapted by Will McCarthy

This notebook visualizes results from the LAX prior elicitation.

Preview on GitHub:
https://htmlpreview.github.io/?https://raw.githubusercontent.com/cogtoolslab/lax/master/analysis/lax-prior-elicitation-drawing-results-visualizer.html?token=ABK3B5AV4P3VTAK3Q3QCTKLA7BUOC

#### Load the raw results CSVs

In [1]:
WORKER_ID, TRIAL_TYPE, TRIAL_INDEX, CONDITION, STIM_URL, RESPONSES = "workerID", "trial_type", "trial_index", "condition", "stimURL", "responses"
CONFIG_NAME = "config_name"


In [2]:
import os, csv, sys
import pandas as pd

results_csv_directory = "../results/csv"
# print("Possible CSV results files to load:")
# print(os.listdir(results_csv_directory))

results_to_visualize = [
    'lax-tower-4-block-unique-silhouettes-categorization-pilot_3.csv',
    'lax-tower-4-block-unique-silhouettes-procedural-pilot_3.csv'
]

result_csv_to_dataframe = {}
for result_csv in results_to_visualize:
    result_csv_to_dataframe[result_csv] = pd.read_csv(os.path.join(results_csv_directory, result_csv))
    result_csv_to_dataframe[result_csv][CONFIG_NAME] = result_csv
    

#### Join the dataframes on the stimuli ID

In [3]:
import pandas as pd
from IPython.display import Image, HTML

def add_numbers_and_space(responses):
    responses = [f"{id}: {response}" for (id, response) in enumerate(responses)]
    responses = '\n'.join(responses)
    return responses

def group_by_stim_url(df, config_name):
    df[config_name] = df[['stimURL','responses']].groupby(['stimURL'])['responses'].transform(lambda responses: add_numbers_and_space(responses))
    df[['stimURL', config_name]].drop_duplicates()
    return df[['stimURL', config_name]]

def group_by_stim_id(df, config_name):
    df[config_name] = df[['stimId','responses']].groupby(['stimId'])['responses'].transform(lambda responses: add_numbers_and_space(responses))
    df[['stimId', config_name]].drop_duplicates()
    return df[['stimId', config_name]]


def path_to_image_html(path):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''

    return '<img src="'+ path + '" style=max-height:124px;"/>'


def stimId_to_s3URL(stimId):
    '''
    tmp
    '''
    return "https://lax-tower-4-block-unique-silhouettes-color-png.s3.amazonaws.com/tower_4_block_unique_silhouette_color_" + str(stimId).zfill(3) + '.png'
    

def stimId_to_tower_html(stimId):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''
    stimURL = stimId_to_s3URL(stimId) 
    return '<img src="'+ stimURL + '" style=max-height:124px;"/>'



### What did human subjects respond for the raw stimuli in each image and condition?

In [4]:
from functools import reduce

grouped_df_list = [group_by_stim_id(df, config_name) for (config_name, df) in result_csv_to_dataframe.items()]
reduced_df = reduce(lambda x, y: pd.merge(x, y, on = 'stimId'), grouped_df_list).drop_duplicates()

HTML(reduced_df.to_html(escape=False ,formatters=dict(stimId=stimId_to_tower_html)).replace("\\n","<br>=======<br><br>"))




,stimId,lax-tower-4-block-unique-silhouettes-categorization-pilot_3.csv,lax-tower-4-block-unique-silhouettes-procedural-pilot_3.csv
0,,"0: {""Q0"":""two red blocks sandwiched between two blue blocks on top and bottom""}=======1: {""Q0"":""blue bottom left of line, 2 red on right stacked, blue top straddle""}=======2: {""Q0"":""it looks like the letter z, or a snake standing up.""}=======3: {""Q0"":""A long, steep Z.""}=======4: {""Q0"":""Four S""}=======5: {""Q0"":""The Letter Z but flipped vertically""}=======6: {""Q0"":""Elongated S ""}=======7: {""Q0"":""an elongated \""s\"" with blue blocks as the top and bottom and red blocks joining the two.""}=======8: {""Q0"":""capital S""}=======9: {""Q0"":""a snack""}","0: {""what-1"":""the top two red blocks"",""where-1"":""directly to the left of the bottom two red blocks""}=======1: {""what-1"":""a blue block"",""where-1"":""horizontal on the bottom left of the middle"",""what-2"":""a red block"",""where-2"":""vertical on top of the right side of the blue block"",""what-3"":""a red block"",""where-3"":""vertically on top of the other red block"",""what-4"":""a blue block"",""where-4"":""hortizontally on the red block with the right side sticking off.""}=======2: {""what-1"":""two blue bricks "",""where-1"":""at the base of the tower"",""what-2"":""four brown bricks "",""where-2"":""on the right blue brick"",""what-3"":""two blue bricks "",""where-3"":""perpendicular to the brown bricks""}=======3: {""what-1"":""the blue block"",""where-1"":""sideways, to the left of the Y axis touching it"",""what-2"":""A red block"",""where-2"":""vertically, on top of the right blue block"",""what-3"":""A red block"",""where-3"":""vertically, on top of the other red block"",""what-4"":""A blue block"",""where-4"":""horizontally, with the left block on top of the red block""}=======4: {""what-1"":""two blue blocks"",""where-1"":""sideways"",""what-2"":""two red blocks"",""where-2"":""vertically""}=======5: {""what-1"":""blue on its side "",""where-1"":""to the left of the vertical line"",""what-2"":""two vertical reds"",""where-2"":""on the right side of the blue"",""what-3"":""one horizontal blue"",""where-3"":""hanging over the opposite side of the first blue""}=======6: {""what-1"":""two blue squares"",""where-1"":""next to each other to the left of the centre"",""what-2"":""four red squares "",""where-2"":""on top of each other and the second blue square"",""what-3"":""two blue squares"",""where-3"":""at the top and to the right of the red boxes""}=======7: {""what-1"":""blue block"",""where-1"":""to the right of center line"",""what-2"":""red block"",""where-2"":""right blue block"",""what-3"":""red block"",""where-3"":""red block"",""what-4"":""blue block"",""where-4"":""left side on top of red block""}=======8: {""what-1"":""blue tiles"",""where-1"":""on bottom""}=======9: {""what-1"":""a blue block horizontally"",""where-1"":""on the bottom of the tower, on the left side of the surface, with the right half of the block aligned with the center of the surface"",""what-2"":""a red block vertically"",""where-2"":""on top of the bottom blue block, on top of its right half"",""what-3"":""a red block vertically"",""where-3"":""on top of the red block below it"",""what-4"":""a blue block horizontally"",""where-4"":""with its left half on top of the red block below it""}"
100,,"0: {""Q0"":""Blue square middle=======Double Red on top of B1 =======Double Red beneath B4""}=======1: {""Q0"":""tetris zigzag""}=======2: {""Q0"":""Blue square standing on a vertical red rectangle offset to one side with a similar red rectangle above offset to the other side.""}=======3: {""Q0"":""lightning bolt shape""}=======4: {""Q0"":""two pieces stuck together pointing opposite directions vertically""}=======5: {""Q0"":""A vertical red under a vertical blue. Another vertical blue is next to the aforementioned vertical blue and a vertical red resides about the second vertical blue.""}=======6: {""Q0"":""A square balanci